In [181]:
import pandas as pd
import numpy as np
import requests
import json
from pandas import json_normalize
from requests.auth import HTTPBasicAuth

from dotenv.main import load_dotenv
import os



pd.options.display.max_seq_items = 20000
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 2000)

In [7]:
#Load the Api Key

from dotenv.main import load_dotenv
import os
load_dotenv()
API_KEY=os.environ["api_key"]

### 0.1 Notes and Thoughts

Hard Filters that are needed for the Dataset are:
- B2C
- Europe 
- (Later industry, first iteration without industry specificat)

All Available fields:
id
name
type
deleted
path
tagline
about
url,website_url,twitter_url,facebook_url,linkedin_url,google_url,crunchbase_url,angellist_url
playmarket_app_id,appstore_app_id
images
employees,employees_latest
industries,sub_industries,corporate_industries,service_industries
technologies
income_streams
growth_stage
traffic_summary
hq_locations
client_focus
revenues
tags
ownerships
delivery_method
launch_year,launch_month
has_promising_founder,has_strong_founder,has_super_founder
total_funding,total_funding_source,last_funding,last_funding_source
company_status,last_updated,last_updated_utc,created_utc
facebook_likes_chart,twitter_tweets_chart,twitter_followers_chart,twitter_favorites_chart
employees_chart
similarweb_3_months_growth_unique,similarweb_3_months_growth_percentile,similarweb_3_months_growth_relative,similarweb_3_months_growth_delta,similarweb_6_months_growth_unique,similarweb_6_months_growth_percentile,similarweb_6_months_growth_relative,similarweb_6_months_growth_delta,similarweb_12_months_growth_unique,similarweb_12_months_growth_percentile,similarweb_12_months_growth_relative,similarweb_12_months_growth_delta

app_3_months_growth_unique,app_3_months_growth_percentile,app_3_months_growth_relative,app_6_months_growth_unique,app_6_months_growth_percentile,app_6_months_growth_relative,app_12_months_growth_unique,app_12_months_growth_percentile,app_12_months_growth_relative,employee_3_months_growth_unique

employee_3_months_growth_percentile,employee_3_months_growth_relative,employee_3_months_growth_delta,employee_6_months_growth_unique,employee_6_months_growth_percentile,employee_6_months_growth_relative,employee_6_months_growth_delta,employee_12_months_growth_unique,employee_12_months_growth_percentile,employee_12_months_growth_relative,employee_12_months_growth_delta
kpi_summary
team
investors
fundings
traffic
similarweb_chart
job_openings
exits
trading_multiple
app_downloads_ios_chart,app_downloads_android_chart,app_downloads_ios_incremental_chart,app_downloads_android_incremental_chart
tech_stack_predictleads
sustainable_development_goals
core_side_value
data_type
pic_number
patents_count


### 0.2 Getting Available Filters for the Post Requests

In [8]:

api_url="https://api.dealroom.co/api/v1/companies/filters"
auth=HTTPBasicAuth(API_KEY, '')

headers = {'Content-Type': 'application/json'}
data={"fields":"hq_locations",
              "limit": 1,
              "offset": 0
             }
response = requests.post(api_url,data=json.dumps(data),auth=auth,headers=headers)
data=response.json()



# 1.1 Building the Core DF

1. Variables for the DF:

1.1: Independent Variables
- Team
- patents (patents_count)
- Round (fundings)
- Investors
- employees (latest, chart)
- Num. Investors (investors)
- (Industry Category) (industries)
- Description Length (about)
- Has Domain (website_url)
- Has Facebook (facebook_url)
- Has Twitter (twitter_url)
- Has LinkedIn (linkedin_url)
- Geographical Region (country, city/-code) (hq_locations)
- Total Funding (total_funding) (in mio)

1.2 Dependent Variables:
- Acquisition (company_status)
- Failure (company_status)
- Public (ownerships)

### Importing the Json file through according API Call:

In [9]:
fields="id,name,about,tagline,investors,industries,website_url,facebook_url,twitter_url,linkedin_url,hq_locations,total_funding,company_status,ownerships,team,fundings,patents_count,employees_latest,employees_chart"
api_url="https://api.dealroom.co/api/v1/companies/bulk"
auth=HTTPBasicAuth(API_KEY, '')
headers = {'Content-Type': 'application/json'}

json_list=[]
next_page_id = ''
k=0

while True:
    data = {
        'form_data': {'must':{'hq_locations': ['Europe'], 'client_focus':['business']}},
        'fields': fields,
        'next_page_id': next_page_id,
        'limit': 100,
        }

    headers = {'Content-Type': 'application/json'}

    response = requests.post(api_url,data=json.dumps(data),auth=auth,headers=headers)

    res=json.loads(response.text)
    json_list.append(res)
    k+=1
    k
    next_page_id = res['next_page_id']

    if next_page_id == None:
        break

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [83]:
with open("first_json.json", "w") as fp:
        json.dump(json_list, fp)

In [12]:
json_list

{'items': [{'id': 2013268,
   'name': 'Loftus Bradford',
   'about': 'Loftus Bradford works with the talent and brands shaping the 21st century.\n\nWe help clients develop pipelines of talent so they can be proactive in their talent planning instead of being reactive to unplanned succession changes.\n\nThe feedback we receive from our clients is simple - they work with us for five main reasons:\n\n1. Boutique approach - Accessibility & a consistent point of contact\n\n2. Zero conflict of interest - We work with only one client per sector – we will not work with our clients’ direct competitors.\n\n3. Global reach - Network of high-potential, diverse, multi-cultural, internationally mobile candidates\n\n4. Succession planning track record - 86% of placements are still with our clients and are on a fast track – we measure this on a rolling 7 year basis.\n\n5. International approach to diversity - 72% of searches we are retained on have a specific diversity objective\n\nSpecialties: Retail

### 1.2 Converting Json to Pandas DF

In [11]:
df_list=[]
for i in range(len(json_list)):
    df_1=pd.json_normalize(json_list[i]["items"],sep="_").drop(columns=["hq_locations","industries"])
    df_1[["country_name","city_name"]]=pd.json_normalize(json_list[0]["items"],"hq_locations")[["country.name","city.name"]]
    df_1["industry_name"]=pd.json_normalize(json_list[0]["items"],"industries")["name"]
    df_list.append(df_1)

In [13]:
df=pd.concat(df_list)
df.to_csv("Raw_Dataset.csv",index=False) #saves the Data to file


In [233]:
df.shape

(196226, 23)

## 1.3 DataCleaning

In [324]:
x=df.isna().sum()
y=df.isna().sum()/len(df)*100
missing_values=pd.concat([x, y], axis=1)
missing_values.columns=["Missing Values","In percent"]
print(missing_values)


                  Missing Values  In percent
id                             0    0.000000
name                           0    0.000000
about                      54159   27.600318
tagline                    57989   29.552149
website_url                    0    0.000000
facebook_url              157205   80.114256
twitter_url               104298   53.151978
linkedin_url               47885   24.402984
total_funding                  0    0.000000
company_status                 0    0.000000
ownerships                     0    0.000000
patents_count                  0    0.000000
employees_latest           56423   28.754090
employees_chart                0    0.000000
investors_items                0    0.000000
investors_total                0    0.000000
team_items                     0    0.000000
team_total                     0    0.000000
fundings_items                 0    0.000000
fundings_total                 0    0.000000
country_name                   0    0.000000
city_name 

In [325]:
df.dropna(subset=["website_url"],inplace=True) #companies without website will be deleted to avoid ghost companies
df.dropna(subset=["company_status"],inplace=True) #companies without status (only 85 so can be dismissed)

### Number of Empty lists as entries

In [326]:
for i,row in missing_values.iterrows():
    try:
        print(i,df[i].apply(lambda x:1 if len(x) == 0 else 0).sum()/len(df)*100,"%")
    except:
        None



name 0.0 %
website_url 0.0 %
company_status 0.0 %
ownerships 58.04072854769501 %
employees_chart 28.75358005565012 %
investors_items 76.68147951851437 %
team_items 63.38609562443305 %
fundings_items 77.07541304414298 %
country_name 0.0 %
industry_name 0.0 %


In [403]:
df_1=df[df["ownerships"].apply(lambda x:False if len(x) == 0 else True)]

In [114]:
df_2=df[df["fundings_items"].apply(lambda x:False if len(x) == 0 else True)]

In [116]:
for i,row in missing_values.iterrows():
    try:
        print(i,df_1[i].apply(lambda x:1 if len(x) == 0 else 0).sum()/len(df)*100,"%")
    except:
        None


for i,row in missing_values.iterrows():
    try:
        print(i,df_2[i].apply(lambda x:1 if len(x) == 0 else 0).sum()/len(df)*100,"%")
    except:
        None       

print(df_1.shape)
print(df_2.shape)

name 0.0 %
website_url 0.0 %
company_status 0.0 %
ownerships 0.0 %
employees_chart 7.5433428801484 %
investors_items 26.505661838900046 %
team_items 9.661308898922671 %
fundings_items 23.30068390529288 %
country_name 0.0 %
industry_name 0.0 %
name 0.0 %
website_url 0.0 %
company_status 0.0 %
ownerships 4.265999408844904 %
employees_chart 3.3160743224649134 %
investors_items 3.7946041808934594 %
team_items 7.788978015145802 %
fundings_items 0.0 %
country_name 0.0 %
industry_name 0.0 %
(82335, 23)
(44984, 23)


When removing only companies with missing fundings we receive a set of 44.984 companies and 7% missing teams (set.1)
When removing only ownerships we receive a set of 82.335 with 9.6% missing teams (set.2)
As the fundings are only relevant when trying to predict probability of follow on fundings we will continue using set.2 and use type of company status as dependent variable

### Further clean Dataset by removing all Empty lists in the team column

In [404]:
df_1=df_1[df_1["team_items"].apply(lambda x:False if len(x) == 0 else True)]
df_1.shape

(63377, 23)

In [119]:
for i,row in missing_values.iterrows():
    try:
        print(i,df_1[i].apply(lambda x:1 if len(x) == 0 else 0).sum()/len(df)*100,"%")
    except:
        None

name 0.0 %
website_url 0.0 %
company_status 0.0 %
ownerships 0.0 %
employees_chart 3.4343053417997615 %
investors_items 21.087419608003017 %
team_items 0.0 %
fundings_items 19.354723634992304 %
country_name 0.0 %
industry_name 0.0 %


### We observe that many of the empty lists for fundings and investors are for bootstrapped startups. It makes sense to distinguish between those where data is missing and those where it simply makes no sense to have data

In [406]:
df_invest=df_1.loc[df_1["investors_items"].apply(lambda x:True if len(x) == 0 else False)]
df_boots=df_invest.loc[df_invest["ownerships"].apply(lambda x:True if ((x[0]=="bootstrapped" and len(x)<=1) or (x[0]=="not known" and len(x)==2)) else False)]

df_boots["investors_items"].values[:]="nan"

df_2=pd.concat([df_1,df_boots])

df_2.drop_duplicates(subset=["id"],keep="last", inplace=True)

,id,name,about,tagline,website_url,facebook_url,twitter_url,linkedin_url,total_funding,company_status,ownerships,patents_count,employees_latest,employees_chart,investors_items,investors_total,team_items,team_total,fundings_items,fundings_total,country_name,city_name,industry_name
15,2537869,mHealthQA,mHealthQA.com is an online platform which make...,Easy Start of CE Marking for mHealth Apps,http://mhealthqa.com,None,None,https://www.linkedin.com/company/mhealthqa/,0.0,operational,[bootstrapped],0,1.0,"[{'date': '2021-07-05', 'value': 2}, {'date': ...",nan,0,"[{'id': 865094, 'name': 'Jesper Lindholt', 'pa...",3,[],0,Denmark,Aarhus,legal
17,3299491,MIT Enterprise Forum Greece,Learn more about the 2022 Startup Competition ...,None,http://mitefgreece.org,https://www.facebook.com/mitenterpriseforumgreece,https://twitter.com/mitefgreece,https://www.linkedin.com/company/mit-enterpris...,0.0,operational,[bootstrapped],0,11.0,"[{'date': '2022-02-08', 'value': 10}, {'date':...",nan,0,"[{'id': 869584, 'name': 'Konstantinos Michanet...",2,[],0,Greece,NaN,transportation
29,3618556,Mothership Missions,The Mothership is an open innovation platform ...,None,http://mothershipmissions.com,https://www.facebook.com/mothershipmissions,https://twitter.com/mothershipsdgs,https://www.linkedin.com/company/mothership-mi...,0.0,operational,[bootstrapped],0,1.0,"[{'date': '2022-02-08', 'value': 1}, {'date': ...",nan,0,"[{'id': 1794764, 'name': 'Bobby Bahov', 'path'...",1,[],0,Netherlands,The Hague,fintech
35,2123015,Movimentar,movimentar GmbH is a social enterprise providi...,Movimentar – Project Management & Data Science,http://movimentar.eu,None,None,https://www.linkedin.com/company/movimentar/,0.0,operational,[bootstrapped],0,20.0,"[{'date': '2020-12-01', 'value': 18}, {'date':...",nan,0,"[{'id': 2975498, 'name': 'Eduardo Wirthmann Fe...",1,[],0,Germany,Bremen,marketing
36,1792080,DBA,"Dba is an audit, certified accounting and cons...","Offers certified accounting, audit and consult...",http://www.fr-dba.com,None,None,https://www.linkedin.com/company/dba-expertise...,0.0,operational,[bootstrapped],0,191.0,"[{'date': '2018-03-31', 'value': 147}, {'date'...",nan,0,"[{'id': 2979031, 'name': 'Marion Petitpoisson'...",2,[],0,France,Paris,marketing


In [437]:
df_fundings=df_2.loc[df_2["fundings_items"].apply(lambda x:True if len(x) == 0 else False)]
df_fundinv=df_fundings.loc[df_fundings["investors_items"].apply(lambda x:True if x == "nan" else False)]
df_shit=df_fundinv.loc[df_fundinv["ownerships"].apply(lambda x:True if ((x[0]=="bootstrapped" and len(x)<=1) or (x[0]=="not known")) else False)]

df_shit["fundings_items"].values[:]="nan"

df_3=pd.concat([df_2,df_shit])

df_3.drop_duplicates(subset=["id"],keep="last", inplace=True)

In [438]:
for i,row in missing_values.iterrows():
    try:
        print(i,df_3[i].apply(lambda x:1 if len(x) == 0 else 0).sum()/len(df)*100,"%")
    except:
        None

for i,row in missing_values.iterrows():
    try:
        print(i,df_3[i].apply(lambda x:1 if len(x) == 0 else 0).sum())
    except:
        None

name 0.0 %
website_url 0.0 %
company_status 0.0 %
ownerships 0.0 %
employees_chart 3.4343053417997615 %
investors_items 4.505009529827852 %
team_items 0.0 %
fundings_items 2.9017561383302923 %
country_name 0.0 %
industry_name 0.0 %
name 0
website_url 0
company_status 0
ownerships 0
employees_chart 6739
investors_items 8840
team_items 0
fundings_items 5694
country_name 0
industry_name 0


Data Seems quite clean to me. No more Data Cleaning Needed 

In [456]:
df_3.loc[df_3["ownerships"].apply(lambda x: True if x[0]=="Private Equity" else False)].shape

(1429, 23)